In [ ]:
import numpy as np
import geopandas as gpd
from shapely.geometry import Point
from tqdm import tqdm

# --- Import your scoring classes here ---
# from aggregate_scoring import (
#     CommunityTransportationOptions,
#     DesirableUndesirableActivities,
#     HousingNeedsCharacteristics,
#     QualityEducation,
#     StableCommunities
# )

# --- Define Grid Parameters ---
lon_min, lon_max = -84.911059, -83.799104
lat_min, lat_max = 33.256500, 34.412590
step = 0.01

lats = np.arange(lat_min, lat_max, step)
lons = np.arange(lon_min, lon_max, step)

records = []

# --- Loop over lat/lon grid ---
for lat in tqdm(lats, desc="Latitude rows"):
    for lon in lons:
        try:
            kwargs = {
                "school_df": school_df,  # load externally
                "state_avg_by_year": state_avg_by_year,
                "census_tract_data": merged_housing_data,  # as dict
                "rural_gdf": rural_gdf,
                "desirable_csv": desirable_csv,
                "grocery_csv": grocery_csv,
                "usda_csv": usda_csv,
                "tract_shapefile": tract_shapefile,
                "undesirable_csv": undesirable_csv,
                "indicators_df": indicators_df,
                "census_tracts_gdf": census_tracts_gdf,
                "stable_community_score": None  # computed internally if not passed
            }

            ct_score = CommunityTransportationOptions(lat, lon, **kwargs).calculate_score()
            dua_score = DesirableUndesirableActivities(lat, lon, **kwargs).calculate_score()
            sc_score = StableCommunities(lat, lon, **kwargs).calculate_score()
            hnc_score = HousingNeedsCharacteristics(lat, lon, stable_community_score=sc_score, **kwargs).calculate_score()
            qe_score = QualityEducation(lat, lon, **kwargs).calculate_score()

            total = ct_score + dua_score + hnc_score + qe_score + sc_score

            records.append({
                "lat": lat,
                "lon": lon,
                "community_transportation_options_score": ct_score,
                "desirable_undesirable_activities_score": dua_score,
                "housing_need_characteristics_score": hnc_score,
                "quality_education_areas_score": qe_score,
                "stable_communities_score": sc_score,
                "total_score": total,
                "geometry": Point(lon, lat)
            })

        except Exception as e:
            print(f"Error at (lat={lat:.4f}, lon={lon:.4f}): {e}")

# --- Export GeoJSON ---
gdf = gpd.GeoDataFrame(records, geometry="geometry", crs="EPSG:4326")
out_path = "../../data/map/final_scoring_dataset.geojson"
gdf.to_file(out_path, driver="GeoJSON")
